In [1]:
import pandas as pd
import os 
import json
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials_gpe-analytics.json"

In [2]:
new_preds = pd.read_json('../data/predictions/final_preds/all_preds.json',orient='records',lines=True)

In [3]:
# Constants 
gcs_folder = 'project-google-arxr-analytics-20240401'
gcs_bucket = 'mm-gpe-data'

# What



1. New twitter/news/insta needs combined with previous table 
CREATE new_updated_table
SELECT * FROM current_table
JOIN new_table 
WHERE current_table.updated_source=Reddit



In [10]:
all_preds = pd.read_json('../data/predictions/final_preds/all_preds.json',orient='records',lines=True)
new_preds = all_preds.loc[~all_preds['Page Type'].isna()].reset_index(drop=True) # Remove all Reddit values 
old_preds = pd.read_json('../data/predictions/final_preds/old-preds.json',orient='records',lines=True)


In [18]:
old_preds = old_preds.loc[old_preds['Page Type'].isna()].reset_index(drop=True)

In [4]:
final_preds = pd.concat([new_preds]).reset_index(drop=True)

In [5]:
final_preds.to_json("../data/predictions/final_preds/05022024_final_preds.json",orient="records",lines=True)

In [5]:
def all_preds_configuration(df):    
    # Change date column to timestamp
    if 'date' in df:
        df['date'] = df['date'].combine_first(df['Date'])
    else:
        df['date']  = df['Date']

    if 'author' in df:
        df['author'] = df['author'].combine_first(df['Author'])
    else:
        df['author'] = df['Author']

    # #Combine author for the news 
    df['author'] = df['author'].combine_first(df['Domain'])
        
    # # Now convert timestamp to string '2022-12-12'
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    # # Twitter
    df['Engagements'] = df.apply(lambda x: x['Twitter Likes'] + x['Twitter Retweets'],axis=1)
    # # Reddit
    if 'redditScore' in df:
       df['engagements'] = df['redditScore'].combine_first(df['Engagements'])
    else:
       df['engagements'] = df['Engagements']
    
    # # Create updated source column
    df['updated_source'] = df.apply(lambda x: x['Page Type'] if x['Page Type'] else 'reddit',axis=1)
    
    #df['link_url'] = df.apply(lambda x: x.uid if x.updated_source!='reddit' else x.url,axis=1)
    df['link_url'] = df['uid']
    bad_colums = [
        "Reach (new)", 
        "Google AR/VR: Companies (Parent - products)",
        "[Chrome AI Snap] Browsers",
        "[Chrome AI Snap] ChatGPT vs Bard",
        "[Chrome AI Snap] Search Engines",
        "Engagements",
        "Date",
        "Author",
        "Google AR/VR: Categories",
        "Google AR/VR: Companies",
        "Google AR/VR: Companies (Parent - products)",
        "Google AR/VR: Events",
        "Google AR/VR: Glasses",
        "Google AR/VR: Headsets"]
    df.drop(columns=bad_colums, inplace=True)
    
    return df 
    
    

In [6]:
final_preds = pd.read_json('../data/predictions/final_preds/05022024_final_preds.json',orient='records',lines=True)

In [7]:
updated_preds = all_preds_configuration(final_preds)
cols = updated_preds.columns
cols.sort_values()
for x in cols:
    print(x)

Unnamed: 0
Query Id
Query Name
Title
text
Full Text
uid
Domain
Sentiment
Emotion
Page Type
Language
Country Code
Continent Code
Continent
Country
Region Code
Region
City Code
Account Type
Assignment
Avatar
Category Details
Checked
City
Display URLs
Expanded URLs
Facebook Author ID
Facebook Comments
Facebook Likes
Facebook Role
Facebook Shares
Facebook Subtype
Full Name
Gender
Hashtags
Impact
Impressions
Instagram Comments
Instagram Followers
Instagram Following
Instagram Likes
Instagram Posts
Interest
Last Assignment Date
Latitude
Location Name
Longitude
Media Filter
Media URLs
Mentioned Authors
Priority
Professions
Resource Id
Short URLs
Starred
Status
Subtype
Tags
Thread Author
Thread Created Date
Thread Entry Type
Thread Id
Thread URL
Total Monthly Visitors
Twitter Author ID
Twitter Channel Role
Twitter Followers
Twitter Following
Twitter Reply Count
Twitter Reply to
Twitter Retweet of
Twitter Retweets
Twitter Likes
Twitter Tweets
Twitter Verified
Reddit Score
Reddit Score Upvote Ra

In [8]:
# Create Table 
client = bigquery.Client()

In [9]:
# Create Table 
client = bigquery.Client()
# Define table name, in format dataset.table_name
table = f'gpe-analytics.{gcs_folder.replace("-","_")}.ar_xr_predictions_updated_pv_3'

# Load data to BQ
job = client.load_table_from_dataframe(updated_preds, table)

# Do Clustering 

In [29]:
# Preprocessing keys 
{"text":"Would you prefer for Apple to adapt to the industry standard of RCS? #fyp #android #apple #iphone #samsung #google #ipager #phone #greenbubble #bluebubble #rcs #androidvsiphone #getthemessage","uid":"https:\/\/www.tiktok.com\/@redirect-to\/video\/7287566836936346912","source":"tiktok","date":"2024-01-04"}.keys()

<>:2: SyntaxWarning: invalid escape sequence '\/'
<>:2: SyntaxWarning: invalid escape sequence '\/'
/var/folders/q7/6w64406173s6jh0blpll135c0000gq/T/ipykernel_8879/3914621348.py:2: SyntaxWarning: invalid escape sequence '\/'
  {"text":"Would you prefer for Apple to adapt to the industry standard of RCS? #fyp #android #apple #iphone #samsung #google #ipager #phone #greenbubble #bluebubble #rcs #androidvsiphone #getthemessage","uid":"https:\/\/www.tiktok.com\/@redirect-to\/video\/7287566836936346912","source":"tiktok","date":"2024-01-04"}.keys()


dict_keys(['text', 'uid', 'source', 'date'])

In [ ]:
# label map keys 
{"source": "Platform", "author": "NULL", "engagements": "NULL", "link": "Video_URL", "uid": "Video_URL", "date": "date", "queryName": "NULL", "query_title": "Video_Title", "subreddit": "NULL", "sentiment": "NULL"}.keys()

dict_keys(['source', 'author', 'engagements', 'link', 'uid', 'date', 'queryName', 'query_title', 'subreddit', 'sentiment'])

In [30]:
preprocessed_data = final_preds[['date','text','uid','source']]


In [ ]:
preprocessed_data.to_json('preprocessed_data.json',orient='records',lines=True)

In [ ]:
os.system(f'gsutil cp preprocessed_data.json gs://{gcs_bucket}/{gcs_folder}/preprocessed/')

Copying file://preprocessed_data.json [Content-Type=application/json]...
\ [1 files][ 44.9 MiB/ 44.9 MiB]                                                
Operation completed over 1 objects/44.9 MiB.                                     


0

In [ ]:
label_map = {'source':'source',
            'author':'author',
            'engagements':'NULL',
            'link':'NULL',
            'uid':'uid',
            'date':'date',
            'queryName':'NULL',
            'query_title':'NULL',
            'subreddit':'NULL',
            'sentiment':'NULL'}


with open('data_label_map.json','w') as f:
    json.dump(label_map,f)
    
os.system(f'gsutil cp data_label_map.json gs://{gcs_bucket}/{gcs_folder}/label_maps/')


Copying file://data_label_map.json [Content-Type=application/json]...
/ [0 files][    0.0 B/  195.0 B]                                                
/ [1 files][  195.0 B/  195.0 B]                                                
Operation completed over 1 objects/195.0 B.                                      


0

In [ ]:
final_preds.to_json('raw_data.json',orient='records',lines=True)
os.system(f'gsutil cp raw_data.json gs://{gcs_bucket}/{gcs_folder}/raw/')


Copying file://raw_data.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1 files][ 1006 MiB/ 1006 MiB]   47.6 MiB/s                                   
Operation completed over 1 objects/1006.7 MiB.                                   


0

# Kick off new pipeline to obtain predictions 

In [31]:
"""SELECT 
    * 
    FROM 
    (
        SELECT 
        DISTINCT LEFT(text, 300) AS text, 
        uid, 
        category_1, 
        probability_1, 
        category_2, 
        probability_2, 
        subreddit_category_1,
        subreddit_probability_1
        FROM 
        `gpe-analytics.project_google_arxr_analytics_20240109.redditnlp_output`
    )  
    LEFT JOIN (SELECT source ,uid,date engagements as NULL from gpe-analytics.project_google_arxr_analytics_20240109.raw_data) USING (uid)
    LEFT JOIN (
        SELECT 
        * 
        FROM 
        `gpe-analytics.project_google_arxr_analytics_20240109.clusters`
    ) USING (uid) 
    LEFT JOIN (SELECT * FROM `gpe-analytics.project_google_arxr_analytics_20240109.custom_nlp_tags`) USING (uid) LEFT JOIN (SELECT text as raw_text, uid,  FROM `gpe-analytics.project_google_arxr_analytics_20240109.raw_data`) USING (uid)"""

'SELECT \n    * \n    FROM \n    (\n        SELECT \n        DISTINCT LEFT(text, 300) AS text, \n        uid, \n        category_1, \n        probability_1, \n        category_2, \n        probability_2, \n        subreddit_category_1,\n        subreddit_probability_1\n        FROM \n        `gpe-analytics.project_google_arxr_analytics_20240109.redditnlp_output`\n    )  \n    LEFT JOIN (SELECT source ,uid,date engagements as NULL from gpe-analytics.project_google_arxr_analytics_20240109.raw_data) USING (uid)\n    LEFT JOIN (\n        SELECT \n        * \n        FROM \n        `gpe-analytics.project_google_arxr_analytics_20240109.clusters`\n    ) USING (uid) \n    LEFT JOIN (SELECT * FROM `gpe-analytics.project_google_arxr_analytics_20240109.custom_nlp_tags`) USING (uid) LEFT JOIN (SELECT text as raw_text, uid,  FROM `gpe-analytics.project_google_arxr_analytics_20240109.raw_data`) USING (uid)'

In [ ]:
"project-google-arxr-analytics-20240401".replace("-","_")

'project_google_arxr_analytics_20240401'

In [ ]:
CREATE TABLE `gpe-analytics.project_google_arxr_analytics_20240401.view_dashboard_final_update` as SELECT 
    * 
    FROM 
    (
        SELECT 
        DISTINCT LEFT(text, 300) AS text, 
        uid, 
        category_1, 
        probability_1, 
        category_2, 
        probability_2, 
        subreddit_category_1,
        subreddit_probability_1
        FROM 
        `gpe-analytics.project_google_arxr_analytics_20240401.redditnlp_output`
    )  
    LEFT JOIN (SELECT updated_source ,link_url,uid,date_,query, tag, llm_label, engagements_ from `gpe-analytics.project_google_arxr_analytics_20240401.ar_xr_predictions_table_update`) USING (uid)
    LEFT JOIN (
        SELECT 
        * 
        FROM 
        `gpe-analytics.project_google_arxr_analytics_20240401.clusters`
    ) USING (uid) 
    LEFT JOIN (SELECT * FROM `gpe-analytics.project_google_arxr_analytics_20240401.custom_nlp_tags`) USING (uid) 

# 2024/01/16 Create Clustering Dashboard 

In [ ]:
all_preds = pd.read_json('../data/predictions/final_preds/all_preds.json',orient='records',lines=True)

In [ ]:
for x in all_preds.columns:
    print(x)

Unnamed: 0
Query Id
Query Name
Date
Title
text
uid
Domain
Sentiment
Emotion
Page Type
Language
Country Code
Continent Code
Continent
Country
Region Code
Region
City Code
Account Type
Assignment
Author
Avatar
Category Details
Checked
City
Display URLs
Expanded URLs
Facebook Author ID
Facebook Comments
Facebook Likes
Facebook Role
Facebook Shares
Facebook Subtype
Full Name
Gender
Hashtags
Impact
Impressions
Instagram Comments
Instagram Followers
Instagram Following
Instagram Likes
Instagram Posts
Interest
Last Assignment Date
Latitude
Location Name
Longitude
Media Filter
Media URLs
Mentioned Authors
Priority
Professions
Resource Id
Short URLs
Starred
Status
Subtype
Tags
Thread Author
Thread Created Date
Thread Entry Type
Thread Id
Thread URL
Total Monthly Visitors
Twitter Author ID
Twitter Channel Role
Twitter Followers
Twitter Following
Twitter Reply Count
Twitter Reply to
Twitter Retweet of
Twitter Retweets
Twitter Likes
Twitter Tweets
Twitter Verified
Reach (new)
Reddit Score
Reddit S

In [ ]:
all_preds['twitter_engagements']